# Generate examples

date: 08 Sept, 2021 <br>

content: <br>
* dataset sample
* data processing with image and histogram

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import copy
import cv2
import sys
import os
import pandas as pd

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline

In [ ]:
def convert_to_color(array):
    mask = np.zeros((array.shape[0], array.shape[1],3), dtype="uint8")
    mask[:,:,0] = array * 255 # for red
    return mask

In [ ]:
figures_path = "/tf/workdir/DA_brain/screenshots/"

## Dataset sample

In [ ]:
sys.path.append(os.path.abspath('../data_utils'))
sys.path.append(os.path.abspath('../models'))

In [ ]:
from data_utils.DataSet2DMixed import DataSet2DMixed

In [ ]:
dataset = DataSet2DMixed("/tf/workdir/data/VS_segm/VS_registered/training", batch_size=16,
                          input_data=["t1", "t2"], input_name=["image", "t2"], shuffle=False,
                         output_data=["vs", "vs_class"], output_name=["vs_out", "vs_class_out"],
                         segm_size=100)
len(dataset)

In [ ]:
data = dataset[0]

In [ ]:
idx = 3
t1 = data[0]["image"][idx]
t2 = data[0]["t2"][idx]
vs = convert_to_color(data[1]["vs_out"][idx])

fig = plt.figure(figsize=(10,10))
plt.imshow(t1, cmap="gray")
plt.imshow(vs, alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "example_t1_vs.png"), bbox_inches='tight', pad_inches=0)

fig = plt.figure(figsize=(10,10))
plt.imshow(t2, cmap="gray")
plt.imshow(vs, alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "example_t2_vs.png"), bbox_inches='tight', pad_inches=0)

In [ ]:

t1 = data[0]["image"][idx]
t2 = data[0]["t2"][idx]

fig = plt.figure(figsize=(10,10))
plt.imshow(t1, cmap="gray")
plt.axis('off')
plt.savefig(os.path.join(figures_path, "example_t1.png"), bbox_inches='tight', pad_inches=0)

fig = plt.figure(figsize=(10,10))
plt.imshow(t2, cmap="gray")
plt.axis('off')
plt.savefig(os.path.join(figures_path, "example_t2.png"), bbox_inches='tight', pad_inches=0)


In [ ]:
vs = data[1]["vs_out"][idx]

fig = plt.figure(figsize=(10,10))
plt.imshow(vs, cmap="gray")
plt.axis('off')
plt.savefig(os.path.join(figures_path, "example_vs.png"), bbox_inches='tight', pad_inches=0)

## Data processing steps

In [ ]:
def load_data_from_folder(data_folder, subfiles=None):
    """
    Load data from data folder
    :param data_folder: path to data folder
    :param subfiles: subfiles in data folder
    :return: list with images as np.arrays
    """

    if subfiles is None:
        subfiles = ["vs_gk_t1_refT1.nii", "vs_gk_t2_refT1.nii", 
                    [f for f in os.listdir(data_folder) if "vs_gk_struc1" in f][0], "vs_gk_statistics.json"]

    path = [os.path.join(data_folder, f) for f in subfiles]
    images = [nib.load(p) for p in path[:-1]]
    return [img.get_fdata() for img in images], pd.read_json(path[-1])

def get_non_zero_slices_segmentation(segmentation):
    """
    Extract all slices of segmentation that are non-zero
    :param segmentation: np.array segmentation mask
    :return: list with non-zero slice indices
    """
    non_zero_index = []
    for idx in range(0, segmentation.shape[2]):
        if np.sum(segmentation[:, :, idx]) != 0:
            non_zero_index.append(idx)
    return non_zero_index

In [ ]:
# load NIFTI images
data, stats = load_data_from_folder("/tf/workdir/data/VS_segm/VS_registered/training/vs_gk_1")

# determine median slice that has segmentation mask
slice_to_vis = int(np.round(np.mean(get_non_zero_slices_segmentation(data[2]))))

In [ ]:
stats

In [ ]:
plt.rcParams.update({'font.size': 22})

In [ ]:
# original
fig = plt.figure(figsize=(10,10))
plt.imshow(data[0][:,:,slice_to_vis], cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t1_vs_orig.png"), bbox_inches='tight', pad_inches=0)


fig = plt.figure(figsize=(10,10))
plt.imshow(data[1][:,:,slice_to_vis], cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t2_vs_orig.png"), bbox_inches='tight', pad_inches=0)


values, edges = np.histogram(data[0][:,:,slice_to_vis], bins=200)
values2, edges2 = np.histogram(data[1][:,:,slice_to_vis], bins=200)
fig = plt.figure(figsize=(10,5))
plt.bar(edges[:-1], values, width=np.diff(edges), align="edge", edgecolor="black", 
        color="blue", label="T1 (source)")
plt.bar(edges2[:-1], values2, width=np.diff(edges2), align="edge", color="red", 
        edgecolor="black", alpha=0.5, label="T2 (target)")
plt.legend()
plt.xlabel("pixel values")
plt.ylabel("occurrence")
plt.savefig(os.path.join(figures_path, "dist_hist_orig.png"), bbox_inches='tight', pad_inches=0)


In [ ]:
values, edges = np.histogram(data[0][:,:,slice_to_vis], bins=200)
values2, edges2 = np.histogram(data[1][:,:,slice_to_vis], bins=200)
fig = plt.figure(figsize=(10,5))
plt.bar(edges[:-1], values, width=np.diff(edges), align="edge", edgecolor="black", 
        color="blue", label="T1 (source)")
plt.bar(edges2[:-1], values2, width=np.diff(edges2), align="edge", color="red", 
        edgecolor="black", alpha=0.5, label="T2 (target)")
plt.vlines(float(stats["t1"]["1st_percentile"]), 0, 50000, color="blue", linestyle="dashed")
plt.vlines(float(stats["t2"]["1st_percentile"]), 0, 50000, color="red", linestyle="dashed")
plt.vlines(float(stats["t1"]["99th_percentile"]), 0, 50000, color="blue", linestyle="dashed")
plt.vlines(float(stats["t2"]["99th_percentile"]), 0, 50000, color="red", linestyle="dashed")
plt.legend()
plt.xlabel("pixel values")
plt.ylabel("occurrence")
plt.savefig(os.path.join(figures_path, "dist_hist_limits.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
# clip
sample = np.clip(data[0][:,:,slice_to_vis], float(stats["t1"]["1st_percentile"]),
                         float(stats["t1"]["99th_percentile"]))
sample2 = np.clip(data[1][:,:,slice_to_vis], float(stats["t2"]["1st_percentile"]),
                         float(stats["t2"]["99th_percentile"]))
fig = plt.figure(figsize=(10,10))
plt.imshow(sample, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t1_vs_afterP1.png"), bbox_inches='tight', pad_inches=0)

fig = plt.figure(figsize=(10,10))
plt.imshow(sample2, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t2_vs_afterP1.png"), bbox_inches='tight', pad_inches=0)

values, edges = np.histogram(sample, bins=200)
values2, edges2 = np.histogram(sample2, bins=200)
fig = plt.figure(figsize=(10,5))
plt.bar(edges[:-1], values, width=np.diff(edges), align="edge", edgecolor="black", 
        color="blue", label="T1 (source)")
plt.bar(edges2[:-1], values2, width=np.diff(edges2), align="edge", color="red", 
        edgecolor="black", alpha=0.5, label="T2 (target)")
plt.legend()
plt.xlabel("pixel values")
plt.ylabel("occurrence")
plt.savefig(os.path.join(figures_path, "dist_hist_afterP1.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
print(float(stats["t1"]["1st_percentile"]), float(stats["t1"]["99th_percentile"]))
print(float(stats["t2"]["1st_percentile"]), float(stats["t2"]["99th_percentile"]))

In [ ]:
# z score normalization
sample = (sample - float(stats["t1"]["mean"])) / float(stats["t1"]["std"])
sample2 = (sample2 - float(stats["t2"]["mean"])) / float(stats["t2"]["std"])
fig = plt.figure(figsize=(10,10))
plt.imshow(sample, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t1_vs_afterP2.png"), bbox_inches='tight', pad_inches=0)

fig = plt.figure(figsize=(10,10))
plt.imshow(sample2, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t2_vs_afterP2.png"), bbox_inches='tight', pad_inches=0)

values, edges = np.histogram(sample, bins=200)
values2, edges2 = np.histogram(sample2, bins=200)
fig = plt.figure(figsize=(10,5))
plt.bar(edges[:-1], values, width=np.diff(edges), align="edge", edgecolor="black", 
        color="blue", label="T1 (source)")
plt.bar(edges2[:-1], values2, width=np.diff(edges2), align="edge", color="red", 
        edgecolor="black", alpha=0.5, label="T2 (target)")
plt.legend()
plt.xlabel("pixel values")
plt.ylabel("occurrence")
plt.savefig(os.path.join(figures_path, "dist_hist_afterP2.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
# range [0,1] volume-based
sample = (sample - float(stats["t1"]["min"])) / (
                float(stats["t1"]["max"]) - float(stats["t1"]["min"]))
sample2 = (sample2 - float(stats["t2"]["min"])) / (
                float(stats["t2"]["max"]) - float(stats["t2"]["min"]))
fig = plt.figure(figsize=(10,10))
plt.imshow(sample, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), cmap="gray", alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t1_vs_afterP3.png"), bbox_inches='tight', pad_inches=0)

fig = plt.figure(figsize=(10,10))
plt.imshow(sample2, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), cmap="gray", alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t2_vs_afterP3.png"), bbox_inches='tight', pad_inches=0)

values, edges = np.histogram(sample, bins=200)
values2, edges2 = np.histogram(sample2, bins=200)
fig = plt.figure(figsize=(10,5))
plt.bar(edges[:-1], values, width=np.diff(edges), align="edge", edgecolor="black", 
        color="blue", label="T1 (source)")
plt.bar(edges2[:-1], values2, width=np.diff(edges2), align="edge", color="red", 
        edgecolor="black", alpha=0.5, label="T2 (target)")
plt.legend()
plt.xlabel("pixel values")
plt.ylabel("occurrence")
plt.savefig(os.path.join(figures_path, "dist_hist_afterP3.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
print(np.max(sample), np.min(sample))
print(np.max(sample2), np.min(sample))

In [ ]:
# range [-1,1]
alpha = -1
beta = 1
sample = ((sample - np.min(sample)) / (np.max(sample) - np.min(sample))) * (beta - alpha) + alpha
sample2 = ((sample2 - np.min(sample2)) / (np.max(sample2) - np.min(sample2))) * (beta - alpha) + alpha

fig = plt.figure(figsize=(10,10))
plt.imshow(sample, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t1_vs_afterP4.png"), bbox_inches='tight', pad_inches=0)

fig = plt.figure(figsize=(10,10))
plt.imshow(sample2, cmap="gray")
plt.imshow(convert_to_color(data[2][:,:,slice_to_vis]), alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "dist_t2_vs_afterP4.png"), bbox_inches='tight', pad_inches=0)

values, edges = np.histogram(sample, bins=200)
values2, edges2 = np.histogram(sample2, bins=200)
fig = plt.figure(figsize=(10,5))
plt.bar(edges[:-1], values, width=np.diff(edges), align="edge", edgecolor="black", 
        color="blue", label="T1 (source)")
plt.bar(edges2[:-1], values2, width=np.diff(edges2), align="edge", color="red", 
        edgecolor="black", alpha=0.5, label="T2 (target)")
plt.legend()
plt.xlabel("pixel values")
plt.ylabel("occurrence")
plt.savefig(os.path.join(figures_path, "dist_hist_afterP4.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
print(np.max(sample), np.min(sample))
print(np.max(sample2), np.min(sample))

## Data Augmentation

In [ ]:
import albumentations as A

In [ ]:
dataset = DataSet2DMixed("/tf/workdir/data/VS_segm/VS_registered/training", batch_size=16,
                          input_data=["t1", "t2"], input_name=["image", "t2"], shuffle=False,
                         output_data=["vs", "vs_class"], output_name=["vs_out", "vs_class_out"],
                         segm_size=100)
len(dataset)

In [ ]:
data = dataset[0]

In [ ]:
idx = 3
t1 = data[0]["image"][idx]
vs = convert_to_color(data[1]["vs_out"][idx])
fig = plt.figure(figsize=(10,10))
plt.imshow(t1, cmap="gray")
plt.imshow(vs, alpha=0.5)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "augm_t1_without.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
augm_methods1 = [A.ShiftScaleRotate(p=1, rotate_limit=20, border_mode=cv2.BORDER_CONSTANT)]
augm_methods2 = [A.VerticalFlip(p=1)]
augm_methods3 = [A.GaussianBlur(p=1, blur_limit=(5, 7))]
augm_methods4 = [A.MedianBlur(p=1, blur_limit=5)]
augm_methods5 = [A.MotionBlur(p=1, blur_limit=(5,7))]

idx = 3
t1 = data[0]["image"][idx]
vs = convert_to_color(data[1]["vs_out"][idx])

In [ ]:
transform = A.Compose(augm_methods1)
transformed = transform(image=t1, mask=vs)
fig = plt.figure(figsize=(10,10))
plt.imshow(transformed["image"], cmap="gray")
plt.imshow(transformed["mask"], alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "augm_t1_shiftscalerotation.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
transform = A.Compose(augm_methods2)
transformed = transform(image=t1, mask=vs)
fig = plt.figure(figsize=(10,10))
plt.imshow(transformed["image"], cmap="gray")
plt.imshow(transformed["mask"], alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "augm_t1_flip.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
transform = A.Compose(augm_methods3)
transformed = transform(image=t1, mask=vs)
fig = plt.figure(figsize=(10,10))
plt.imshow(transformed["image"], cmap="gray")
plt.imshow(transformed["mask"], alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "augm_t1_gaussian.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
transform = A.Compose(augm_methods4)
transformed = transform(image=t1, mask=vs)
fig = plt.figure(figsize=(10,10))
plt.imshow(transformed["image"], cmap="gray")
plt.imshow(transformed["mask"], alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "augm_t1_median.png"), bbox_inches='tight', pad_inches=0)

In [ ]:
transform = A.Compose(augm_methods5)
transformed = transform(image=t1, mask=vs)
fig = plt.figure(figsize=(10,10))
plt.imshow(transformed["image"], cmap="gray")
plt.imshow(transformed["mask"], alpha=0.4)
plt.axis('off')
plt.savefig(os.path.join(figures_path, "augm_t1_motion.png"), bbox_inches='tight', pad_inches=0)